In [10]:
import os
import json

# directory where JSON files are located
directory = os.getcwd()

# get all files in the current working directory
files = os.listdir(directory)

# filter out only JSON files
json_files = [file for file in files if file.endswith(".json")]

# initialize a dictionary to store the data from all the JSON files
data = {}

# helper function to merge two dictionaries with nested lists


def merge_dicts(dict1, dict2):
    for key, value in dict2.items():
        if key in dict1 and isinstance(value, (list, dict)):
            # if the key exists in both dictionaries and the value is a list or dictionary, recursively merge the nested data
            if isinstance(value, list):
                # if the value is a list, merge the lists
                dict1[key] = dict1[key] + value
            else:
                # if the value is a dictionary, recursively merge the dictionaries
                dict1[key] = merge_dicts(dict1[key], value)
        else:
            # otherwise, just add the key-value pair from dict2 to dict1
            dict1[key] = value
    return dict1


# load and parse each JSON file, and merge the data into the dictionary
for file in json_files:
    with open(os.path.join(directory, file)) as json_file:
        file_data = json.load(json_file)
        data = merge_dicts(data, file_data)

if not os.path.exists("output"):
    os.makedirs("output")

# save the merged data to a new JSON file
with open("output/merged_data.json", "w") as outfile:
    json.dump(data, outfile)

data = json.load(open(os.path.join(directory, "output/merged_data.json")))
    



In [11]:
def score_path(path,trails,tree):
    score = 0
    for trail in path:
        score +=trails[trail]["distance"]
    return score
    

In [12]:
def decode(encoded):
    path = []
    n = [0, 0]
    j = 0
    for i in range(len(encoded)):
        k = ord(encoded[i]) - 0x30
        n[j] = (n[j] << 5) | (k & 0x1f)
        if k & 0x20:
            n[j] = (~(n[j] >> 1) if n[j] & 0x01 else n[j] >> 1) * 1e-7
            if j == 1:
                latLon = path[-1] if len(path) > 0 else (0, 0)
                path.append((latLon[0] + n[0], latLon[1] + n[1]))
            j = (j + 1) % 2
            n[j] = 0
    return (path[0], path[-1], path)

In [13]:
for trail in data["trails"].values():
    # print(trail)
    startPos, endPos, path = decode(trail.pop("encoded"))
    trail["start_latlng"] = (round(startPos[0], 6), round(startPos[1], 6))
    trail["end_latlng"] = (round(endPos[0], 6), round(endPos[1], 6))
    new_path = []
    for x,y in path:
        new_path.append((round(x, 6), round(y, 6)))
    trail["path"] = new_path


In [14]:
import numpy as np
print(len(data["nodes"]))

good_nodes = []

nodes_exemptions = ["Kežmarské Žľaby","Tatranské Matliare","Štart","Tatranská Lomnica, Grand","Starý Smokovec","Štrbské Pleso, rázcestie pred Heliosom","Liečebný Dom, Solisko, rázc.","Štrbské Pleso","Nadbanské","Podbanské"]
for node in data["nodes"].values():
    if node["name"] in ["Lomnické sedlo","Lomnická veža"]:
        continue
    if node["name"] in nodes_exemptions:
        good_nodes.append(node)
        continue
    if node["region"]!="Tatry":
        for trail in data["trails"].values():
            if trail["start_latlng"] == (node["latitude"], node["longitude"]):
                for node2 in data["nodes"].values():
                    if trail["end_latlng"] == (node2["latitude"], node2["longitude"]):
                        if node2["region"]=="Tatry":
                            good_nodes.append(node)
            if trail["end_latlng"] == (node["latitude"], node["longitude"]):
                for node2 in data["nodes"].values():
                    if trail["start_latlng"] == (node2["latitude"], node2["longitude"]):
                        if node2["region"]=="Tatry":
                            good_nodes.append(node)
    else:
        good_nodes.append(node)

print(len(good_nodes))

# data["nodes"] = {key: value for (key, value) in data["nodes"].items() if value["region"] == "Tatry"}
# print(len(data["nodes"]))
data["nodes"] = {i: value for i, value in enumerate(good_nodes)}
print(len(data["nodes"]))

start_points = ["Siwa Polana", "Polana Biały Potok", "Kiry", "Staników Żleb", "Nędzówka, las",
 "Gronik", "Za Bramką", "Dolina Strążyska, parking","Dolina ku Dziurze", "Dolina Białego",
 "Kuźnicka Polana", "Kuźnice", "Jaszczurówka","Toporowa Cyrhla", "Brzeziny","Dolina Filipka",
 "Wierch Poroniec", "Lysá Poľana, parking", "Palenica Białczańska","Tatranská Javorina", "Strednica",
 "Tatranská Kotlina", "Kežmarské Žľaby","Kežmarská Biela voda, bus","Tatranské Matliare","Tatranská Lomnica",
 "Tatranská Lesná","Starý Smokovec","Tatranské Zruby","Tatranská Polianka","Vyšné Hágy","Popradské Pleso, zastavka",
 "Štrbské Pleso","Biely Váh, rázc.","Bývalá Važecká chata","Tri studničky","Podbanské","Hrdovo","Pribylina, Esperanto",
 "Pod Holým vrchom","Bobrovecká Vápenica","Biela skala, horáreň","Prte","Brestová, skanzen","Pod Spálenou","Oravice"]
 
start_points_ids = []

for i,node in enumerate(data["nodes"].values()):
    node["id"]=i
    if node["name"] in start_points:
        start_points_ids.append(i)

print(start_points_ids)


matrix = np.full((len(data["nodes"]), len(data["nodes"])),np.inf)
for node in data["nodes"].values():
    for trail in data["trails"].values():
        if trail["start_latlng"] == (node["latitude"], node["longitude"]):
            for node2 in data["nodes"].values():
                if trail["end_latlng"] == (node2["latitude"], node2["longitude"]):
                    matrix[node["id"], node2["id"]] = trail["start_end_time"]
                    matrix[node2["id"], node["id"]] = trail["end_start_time"]
        if trail["end_latlng"] == (node["latitude"], node["longitude"]):
            for node2 in data["nodes"].values():
                if trail["start_latlng"] == (node2["latitude"], node2["longitude"]):
                    matrix[node["id"], node2["id"]] = trail["start_end_time"]
                    matrix[node2["id"], node["id"]] = trail["end_start_time"]

# print(matrix)
np.set_printoptions(threshold=np.inf, linewidth=np.inf)
with open("matrix.txt", 'w') as f:
    f.write(np.array2string(matrix, separator=', '))

        

764
380
380
[2, 3, 7, 9, 12, 15, 22, 31, 35, 60, 62, 69, 87, 105, 111, 112, 115, 118, 140, 149, 176, 188, 199, 202, 204, 213, 216, 221, 229, 236, 249, 252, 276, 278, 319, 346, 354, 355, 358, 360, 362, 372, 377]


In [15]:
start_points = ["Siwa Polana", "Polana Biały Potok", "Kiry", "Staników Żleb", "Nędzówka, las",
 "Gronik", "Za Bramką", "Dolina Strążyska, parking","Dolina ku Dziurze", "Dolina Białego",
 "Kuźnicka Polana", "Kuźnice", "Jaszczurówka","Toporowa Cyrhla", "Brzeziny","Dolina Filipka",
 "Wierch Poroniec", "Lysá Poľana, parking", "Palenica Białczańska","Tatranská Javorina", "Strednica",
 "Tatranská Kotlina", "Kežmarské Žľaby","Kežmarská Biela voda, bus","Tatranské Matliare","Tatranská Lomnica",
 "Tatranská Lesná","Starý Smokovec","Tatranské Zruby","Tatranská Polianka","Vyšné Hágy","Popradské Pleso, zastavka",
 "Štrbské Pleso","Biely Váh, rázc.","Bývalá Važecká chata","Tri studničky","Podbanské","Hrdovo","Pribylina, Esperanto",
 "Pod Holým vrchom","Bobrovecká Vápenica","Biela skala, horáreň","Prte","Brestová, skanzen","Pod Spálenou","Oravice"]

for point in start_points:
    point=point.encode("UTF-16")

In [16]:
print(np.count_nonzero(matrix))
print(len(data["trails"].values()))


144400
971


In [17]:
nodes = data["nodes"].values()
good_list = []
for node in nodes:
    good_list.append(node)
# print(good_list)
route = [204, 191, 253, 191, 316, 280, 241, 184, 327, 253, 251, 236, 87, 77, 90, 104, 42, 55, 89, 36, 121, 123, 121, 123, 36, 91, 36, 89, 55, 42, 104, 90, 104, 85, 117, 85, 104, 42, 55, 25, 102, 40, 102, 82, 48, 18, 8, 18, 6, 2, 236, 149, 354, 349, 378, 365, 369, 366, 248, 175, 232, 313, 217, 287, 217, 342, 238, 139, 217, 342, 238, 139, 217, 313, 232, 287, 232, 287, 217, 342, 238, 331, 147, 331, 239, 138, 246, 126, 208, 126, 208, 145, 249, 87, 77, 29, 26, 95, 110, 95, 26, 95, 110, 95, 105, 252, 335, 172, 340, 328, 172, 335, 252, 355, 370, 357, 359, 357, 370, 355, 118, 73, 64, 73, 108, 69, 62, 92, 76, 35, 354, 349, 378, 365, 369, 168, 153, 129, 332, 282, 244, 282, 244, 154, 244, 154, 145, 249, 216, 339, 340, 172, 335, 252, 319, 223, 319, 188, 265, 167, 311, 199, 115, 113, 23, 84, 23, 122, 50, 78, 50, 122, 29, 122, 50, 58, 109, 74, 61, 66, 61, 114, 101, 114, 101, 116, 80, 117, 80, 117, 85, 117, 80, 123, 121, 36, 89, 36, 123, 36, 89, 36, 123, 121, 36, 91, 120, 19, 2, 354, 349, 373, 375, 362, 221, 158, 263, 47, 291, 47, 263, 150, 242, 211, 198, 310, 301, 338, 52, 338, 267, 305, 292, 264, 295, 32, 52, 338, 52, 183, 52, 338, 301, 308, 181, 308, 301, 310, 301, 308, 301, 310, 173, 207, 266, 207, 254, 281, 321, 266, 179, 198, 179, 233, 211, 233, 179, 266, 321, 266, 207, 266, 321, 306, 159, 306, 189, 185, 189, 306, 159, 255, 368, 359, 362, 69, 96, 100, 96, 69, 188, 229, 221, 158, 263, 150, 242, 322, 242, 150, 286, 183, 291, 47, 263, 158, 221, 62, 92, 94, 30, 47, 291, 286, 183, 52, 338, 301, 308, 181, 302, 210, 45, 210, 45, 107, 45, 107, 45, 51, 68, 72, 63, 39, 63, 39, 93, 59, 72, 59, 93, 39, 93, 59, 72, 68, 54, 68, 72, 63, 39, 93, 27, 35, 140, 309, 209, 273, 278, 9, 8, 17, 0, 14, 4, 67, 65, 16, 20, 75, 28, 46, 83, 46, 83, 46, 28, 88, 38, 88, 28, 75, 20, 11, 81, 285, 81, 11, 20, 16, 75, 28, 75, 28, 75, 16, 65, 1, 15, 362, 375, 373, 296, 141, 124, 243, 157, 269, 218, 307, 175, 232, 287, 217, 342, 234, 323, 318, 160, 318, 323, 234, 342, 217, 287, 300, 287, 232, 313, 341, 165, 273, 278, 35, 76, 70, 30, 47, 263, 158, 335, 172, 335, 158, 221, 213, 127, 132, 222, 186, 127, 213, 229, 182, 231, 303, 339, 216, 22, 6, 2, 377, 371, 351, 10, 377, 111, 24, 35, 60, 99, 43, 84, 23, 84, 43, 113, 23, 84, 43, 99, 43, 113, 23, 122, 50, 78, 50, 122, 23, 84, 23, 113, 43, 113, 115, 276, 333, 169, 220, 169, 324, 169, 220, 169, 333, 228, 240, 184, 226, 203, 227, 203, 226, 185, 189, 306, 321, 281, 254, 207, 254, 207, 173, 310, 198, 310, 173, 214, 173, 207, 254, 330, 181, 294, 247, 294, 181, 330, 212, 330, 181, 308, 247, 294, 181, 302, 210, 295, 264, 192, 264, 292, 256, 247, 294, 181, 302, 178, 294, 247, 308, 301, 310, 173, 207, 254, 281, 306, 312, 193, 189, 185, 241, 185, 226, 203, 226, 185, 241, 184, 226, 184, 327, 184, 240, 268, 240, 184, 241, 280, 241, 185, 226, 184, 240, 268, 257, 268, 257, 268, 240, 228, 177, 327, 184, 241, 280, 316, 191, 204, 87, 77, 29, 122, 50, 78, 106, 78, 50, 78, 106, 109, 58, 41, 58, 109, 106, 78, 64, 31, 362, 359, 357, 170, 125, 131, 125, 170, 125, 288, 46, 83, 46, 28, 75, 16, 65, 1, 16, 75, 67, 65, 16, 65, 16, 75, 20, 11, 81, 285, 352, 13, 21, 11, 81, 285, 81, 11, 20, 75, 28, 46, 288, 262, 155, 262, 288, 125, 170, 368, 170, 357, 359, 368, 170, 368, 255, 159, 255, 368, 170, 357, 359, 368, 170, 357, 370, 355, 7, 14, 4, 67, 75, 28, 46, 288, 262, 155, 262, 161, 159, 306, 321, 306, 189, 185, 189, 306, 321, 281, 306, 189, 185, 226, 184, 226, 203, 227, 205, 329, 304, 329, 304, 199, 236, 251, 177, 327, 184, 241, 185, 189, 306, 321, 281, 254, 281, 306, 189, 185, 241, 280, 241, 280, 241, 184, 226, 203, 230, 205, 227, 203, 230, 151, 164, 151, 164, 151, 164, 151, 230, 205, 329, 304, 199, 221, 158, 263, 47, 291, 286, 150, 286, 150, 242, 150, 286, 291, 286, 183, 52, 338, 52, 32, 33, 63, 39, 93, 27, 35, 362, 375, 373, 296, 141, 124, 243, 157, 269, 243, 157, 144, 366, 248, 160, 248, 366, 144, 307, 144, 307, 144, 366, 369, 365, 378, 360, 69, 108, 101, 114, 61, 74, 103, 74, 61, 114, 101, 108, 101, 114, 61, 66, 53, 116, 53, 66, 61, 114, 101, 116, 80, 116, 53, 116, 101, 108, 73, 118, 216, 339, 303, 195, 188, 372, 350, 364, 350, 364, 350, 372, 111, 24, 98, 40, 102, 25, 55, 89, 36, 91, 100, 12, 249, 145, 208, 237, 166, 142, 166, 237, 208, 126, 190, 152, 244, 282, 163, 133, 215, 136, 154, 145, 249, 229, 258, 293, 258, 293, 258, 293, 258, 293, 193, 293, 258, 293, 258, 229, 2, 19, 91, 36, 121, 96, 69, 377, 371, 351, 352, 13, 352, 285, 81, 285, 352, 351, 371, 363, 180, 289, 180, 363, 358, 229, 182, 231, 325, 322, 233, 179, 266, 179, 233, 322, 242, 150, 263, 158, 335, 252, 249, 145, 154, 136, 133, 215, 162, 270, 250, 196, 361, 376, 374, 376, 361, 376, 374, 376, 361, 196, 361, 376, 379, 372, 9, 8, 17, 0, 17, 48, 97, 34, 97, 120, 91, 19, 120, 97, 34, 97, 34, 97, 48, 17, 0, 14, 4, 67, 75, 67, 65, 1, 15, 278, 273, 209, 290, 343, 345, 326, 171, 326, 272, 326, 171, 272, 326, 345, 343, 298, 343, 345, 343, 298, 343, 345, 343, 290, 209, 309, 277, 206, 317, 275, 314, 201, 134, 224, 134, 197, 146, 156, 146, 156, 171, 326, 272, 326, 345, 347, 345, 347, 345, 343, 290, 209, 309, 140, 112, 92, 76, 70, 27, 93, 39, 63, 72, 59, 79, 59, 72, 63, 72, 68, 54, 86, 54, 68, 72, 59, 79, 59, 72, 63, 33, 39, 63, 39, 33, 39, 63, 39, 93, 27, 93, 27, 70, 76, 70, 30, 70, 76, 35, 229, 258, 293, 193, 293, 258, 293, 193, 189, 185, 226, 184, 241, 280, 316, 191, 277, 309, 209, 309, 209, 273, 165, 341, 313, 217, 313, 341, 336, 271, 334, 284, 334, 284, 274, 336, 271, 334, 284, 274, 284, 334, 271, 336, 274, 245, 314, 275, 317, 206, 277, 309, 140, 176, 283, 165, 341, 336, 341, 336, 341, 165, 273, 165, 341, 313, 217, 287, 232, 287, 300, 287, 232, 175, 307, 175, 232, 313, 217, 287, 232, 287, 217, 139, 238, 342, 234, 323, 279, 152, 244, 282, 332, 337, 137, 315, 137, 337, 332, 129, 153, 168, 130, 160, 318, 148, 130, 148, 315, 148, 315, 137, 337, 332, 129, 332, 235, 270, 250, 196, 361, 376, 361, 376, 379, 372, 360, 378, 365, 353, 297, 153, 137, 315, 148, 315, 137, 153, 297, 367, 379, 372, 249, 128, 143, 135, 128, 249, 362, 375, 373, 296, 141, 225, 348, 375, 373, 296, 373, 375, 362, 278, 176, 249, 142, 166, 237, 208, 237, 166, 237, 208, 237, 246, 138, 246, 237, 208, 145, 154, 244, 282, 244, 152, 279, 323, 318, 148, 130, 148, 130, 168, 153, 297, 367, 372, 213, 200, 132, 222, 132, 127, 132, 222, 186, 127, 132, 200, 213, 346, 276, 229, 258, 151, 258, 293, 193, 312, 306, 281, 254, 330, 212, 330, 254, 207, 173, 214, 173, 310, 198, 179, 233, 322, 242, 150, 263, 47, 30, 94, 92, 112, 149, 204, 213, 200, 132, 127, 186, 222, 132, 222, 186, 147, 186, 222, 132, 222, 186, 147, 331, 239, 190, 126, 246, 138, 239, 279, 323, 234, 342, 238, 331, 147, 186, 127, 132, 200, 213, 355, 358, 319, 199, 204, 202, 140, 278, 276, 319, 278, 273, 209, 290, 219, 347, 219, 290, 209, 309, 140, 3, 10, 21, 13, 5, 13, 21, 10, 351, 371, 351, 371, 363, 180, 370, 355, 372, 379, 376, 374, 376, 361, 376, 374, 376, 374, 376, 374, 376, 361, 196, 250, 129, 153, 168, 130, 160, 248, 175, 307, 218, 307, 175, 248, 366, 369, 365, 353, 350, 372, 149, 320, 149, 199, 304, 324, 194, 324, 304, 324, 194, 324, 304, 329, 205, 230, 203, 227, 203, 227, 205, 230, 203, 226, 184, 241, 185, 189, 306, 159, 255, 368, 170, 357, 359, 368, 170, 357, 370, 357, 170, 125, 170, 125, 131, 125, 170, 357, 359, 362, 176, 259, 176, 236, 346, 202, 299, 202, 35, 27, 70, 76, 70, 30, 47, 291, 286, 291, 183, 52, 32, 295, 210, 45, 107, 45, 51, 71, 51, 71, 51, 71, 51, 68, 72, 63, 39, 33, 63, 72, 59, 93, 39, 93, 27, 35, 9, 8, 17, 48, 82, 37, 82, 37, 82, 102, 25, 102, 40, 98, 24, 35, 60, 49, 60, 199, 304, 324, 194, 324, 304, 329, 205, 230, 205, 227, 203, 230, 203, 227, 205, 230, 203, 226, 185, 241, 184, 241, 280, 174, 260, 206, 317, 275, 314, 201, 314, 245, 314, 201, 156, 146, 197, 134, 224, 134, 201, 156, 146, 197, 146, 156, 171, 326, 272, 326, 345, 326, 171, 156, 171, 326, 171, 272, 326, 171, 326, 345, 343, 298, 343, 290, 209, 309, 140, 118, 73, 108, 101, 114, 106, 78, 64, 73, 118, 229, 182, 231, 325, 322, 325, 231, 182, 325, 231, 182, 229, 176, 283, 165, 273, 209, 309, 277, 206, 317, 275, 187, 275, 317, 206, 317, 275, 317, 206, 277, 191, 204, 229, 258, 293, 193, 312, 306, 159, 161, 159, 255, 368, 170, 357, 170, 125, 288, 262, 161, 344, 161, 159, 255, 368, 170, 357, 370, 357, 359, 362, 346, 261, 346]
output = []
for element in route:
    output.append((good_list[element]['latitude'],good_list[element]['longitude']))
formatted_output = ""
for element in output:
    formatted_output+=f"{element[0]},{element[1]};"
# print(formatted_output)
fancy_output = [output[0]]
for i in range(1,len(output)):
    fancy_output.append(output[i])
    for trail in data["trails"].values():
        if trail["start_latlng"] == output[i-1] and trail["end_latlng"] == output[i]:
            fancy_output[-1:-1] = list(trail["path"])
            break
        elif trail["end_latlng"] == output[i-1] and trail["start_latlng"] == output[i]:
            # print(trail["path"])
            fancy_output[-1:-1] = list(reversed(trail["path"]))
            break
    # else:
    #     fancy_output[-1:-1] = [(0,0)]
# print(fancy_output)

fancy_string = '''<?xml version="1.0" encoding="utf-8" standalone="yes"?>
<gpx version="1.1">
<trk>
  <name>bo2</name>
  <trkseg>
'''
for point in fancy_output:
    # if(point==(0,0)):
    #     fancy_string+="</trkseg>\n<trkseg>\n"
    # else:
    fancy_string+=f'<trkpt lat="{point[0]}" lon="{point[1]}"></trkpt>\n'
fancy_string+='''  </trkseg>
</trk>
</gpx>

'''

with open("output.gpx", 'w') as f:
    f.write(fancy_string)


In [18]:
# print(data["trails"].values())